In [69]:
import requests
from tqdm import tqdm_notebook as tqdm
import json
from collections import OrderedDict
import pprint
import numpy as np
import os

# 都道府県コードリストの取得
area_code = []
def get_areacode():
    global area_code
    for j in range(1,48):
        if j in range(10):
            area_code.append("0"+str(j))
        else:
            area_code.append(str(j))
    print("都道府県コードリスト取得完了。都道府県コードリストの長さは",len(area_code))

# 市町村コードリストの取得
citycode_list = []
def get_citycode():
    global citycode_list
    for area in tqdm(area_code, desc="市町村コード取得"):
        r = requests.get("http://www.land.mlit.go.jp/webland/api/CitySearch", {"area": area})
        for data in r.json()["data"]:
                    if("id" in data):
                        citycode_list.append(data["id"])
    print("市町村コードリスト取得完了. 市町村コードリストの長さは",len(citycode_list))

#時間かかるやつ
output = {}
def get_pricelist():
    global output
    for area in tqdm(citycode_list, desc="場所ごとの地価平均取得"):
        pricedict_in_area = {}
        for i in tqdm(range(2006, 2019), desc="一年ごとにとってきています"):
            payload = {"from": str(i) + '1', "to":str(i) + "4", "city": area}
            r = requests.get('http://www.land.mlit.go.jp/webland/api/TradeListSearch', params=payload)
            area_price = []
            for data in r.json()["data"]:
                if("PricePerUnit" in data):
                    area_price.append(data["PricePerUnit"])
            mean_price = np.mean([int(price) for price in area_price])
            median_price = np.median([int(price) for price in area_price])
            standard_deviation_price = np.std([int(price) for price in area_price])
            pricedict_in_area[i] = {"mean":mean_price, "median": median_price, "std": standard_deviation_price}
            print("市町村コード",area,"の",i,"年のデータを取得完了")
        output[area] = pricedict_in_area
        print("市町村コード",area,"の地価取得完了")
    print("地価取得完了. 長さ=",len(output))

# JSON書き出し
def write_json():
    with open(os.getcwd()+'/output.json', mode='w') as f:
        json.dump(output, f, indent=3, sort_keys=True)
    print("書き出し終了！")

In [70]:
if __name__ == "__main__":
    get_areacode()
    get_citycode()
    get_pricelist()
    write_json()

都道府県コードリスト取得完了。都道府県コードリストの長さは 47


市町村コードリスト取得完了. 市町村コードリストの長さは 1922


市町村コード 01100 の 2006 年のデータを取得完了
市町村コード 01100 の 2007 年のデータを取得完了
市町村コード 01100 の 2008 年のデータを取得完了
市町村コード 01100 の 2009 年のデータを取得完了
市町村コード 01100 の 2010 年のデータを取得完了
市町村コード 01100 の 2011 年のデータを取得完了
市町村コード 01100 の 2012 年のデータを取得完了
市町村コード 01100 の 2013 年のデータを取得完了
市町村コード 01100 の 2014 年のデータを取得完了
市町村コード 01100 の 2015 年のデータを取得完了
市町村コード 01100 の 2016 年のデータを取得完了
市町村コード 01100 の 2017 年のデータを取得完了
市町村コード 01100 の 2018 年のデータを取得完了
市町村コード 01100 の地価取得完了


市町村コード 01101 の 2006 年のデータを取得完了
市町村コード 01101 の 2007 年のデータを取得完了
市町村コード 01101 の 2008 年のデータを取得完了
市町村コード 01101 の 2009 年のデータを取得完了
市町村コード 01101 の 2010 年のデータを取得完了
市町村コード 01101 の 2011 年のデータを取得完了
市町村コード 01101 の 2012 年のデータを取得完了
市町村コード 01101 の 2013 年のデータを取得完了
市町村コード 01101 の 2014 年のデータを取得完了
市町村コード 01101 の 2015 年のデータを取得完了
市町村コード 01101 の 2016 年のデータを取得完了
市町村コード 01101 の 2017 年のデータを取得完了
市町村コード 01101 の 2018 年のデータを取得完了
市町村コード 01101 の地価取得完了


市町村コード 01102 の 2006 年のデータを取得完了
市町村コード 01102 の 2007 年のデータを取得完了
市町村コード 01102 の 2008 年のデータを取得完了
市町村コード 01102 の 2009 年のデータを取得完了
市町村コード 01102 の 2010 年のデータを取得完了
市町村コード 01102 の 2011 年のデータを取得完了
市町村コード 01102 の 2012 年のデータを取得完了
市町村コード 01102 の 2013 年のデータを取得完了
市町村コード 01102 の 2014 年のデータを取得完了
市町村コード 01102 の 2015 年のデータを取得完了
市町村コード 01102 の 2016 年のデータを取得完了
市町村コード 01102 の 2017 年のデータを取得完了
市町村コード 01102 の 2018 年のデータを取得完了
市町村コード 01102 の地価取得完了


...

市町村コード 01106 の 2006 年のデータを取得完了
市町村コード 01106 の 2007 年のデータを取得完了
市町村コード 01106 の 2008 年のデータを取得完了
市町村コード 01106 の 2009 年のデータを取得完了
市町村コード 01106 の 2010 年のデータを取得完了
市町村コード 01106 の 2011 年のデータを取得完了
市町村コード 01106 の 2012 年のデータを取得完了
市町村コード 01106 の 2013 年のデータを取得完了
市町村コード 01106 の 2014 年のデータを取得完了
市町村コード 01106 の 2015 年のデータを取得完了
市町村コード 01106 の 2016 年のデータを取得完了
市町村コード 01106 の 2017 年のデータを取得完了
市町村コード 01106 の 2018 年のデータを取得完了
市町村コード 01106 の地価取得完了


市町村コード 01107 の 2006 年のデータを取得完了
市町村コード 01107 の 2007 年のデータを取得完了
市町村コード 01107 の 2008 年のデータを取得完了
市町村コード 01107 の 2009 年のデータを取得完了
市町村コード 01107 の 2010 年のデータを取得完了
市町村コード 01107 の 2011 年のデータを取得完了
市町村コード 01107 の 2012 年のデータを取得完了
市町村コード 01107 の 2013 年のデータを取得完了
市町村コード 01107 の 2014 年のデータを取得完了
市町村コード 01107 の 2015 年のデータを取得完了
市町村コード 01107 の 2016 年のデータを取得完了
市町村コード 01107 の 2017 年のデータを取得完了
市町村コード 01107 の 2018 年のデータを取得完了
市町村コード 01107 の地価取得完了


市町村コード 01108 の 2006 年のデータを取得完了
市町村コード 01108 の 2007 年のデータを取得完了
市町村コード 01108 の 2008 年のデータを取得完了
市町村コード 01108 の 2009 年のデータを取得完了
市町村コード 01108 の 2010 年のデータを取得完了
市町村コード 01108 の 2011 年のデータを取得完了
市町村コード 01108 の 2012 年のデータを取得完了
市町村コード 01108 の 2013 年のデータを取得完了
市町村コード 01108 の 2014 年のデータを取得完了
市町村コード 01108 の 2015 年のデータを取得完了
市町村コード 01108 の 2016 年のデータを取得完了
市町村コード 01108 の 2017 年のデータを取得完了
市町村コード 01108 の 2018 年のデータを取得完了
市町村コード 01108 の地価取得完了


市町村コード 01109 の 2006 年のデータを取得完了
市町村コード 01109 の 2007 年のデータを取得完了
市町村コード 01109 の 2008 年のデータを取得完了
市町村コード 01109 の 2009 年のデータを取得完了
市町村コード 01109 の 2010 年のデータを取得完了
市町村コード 01109 の 2011 年のデータを取得完了
市町村コード 01109 の 2012 年のデータを取得完了
市町村コード 01109 の 2013 年のデータを取得完了
市町村コード 01109 の 2014 年のデータを取得完了
市町村コード 01109 の 2015 年のデータを取得完了


KeyboardInterrupt: 